# Example Firefly usage

This builds off of the [minimal example](https://firefly.rcs.northwestern.edu/docs/data_reader/minimal_example.html) from our docs and the new [data selection example](https://github.com/ageller/Firefly/blob/main/src/firefly/ntbks/selecting_data.ipynb) from our GitHub repo.


As an example, we will generate a random sphere of data with a randomized attribute field.  We will load these data into Firefly.

This notebook also demonstrates how to retrieve data that you selected by hand in the Firefly app and then use those data with your notebook. 

In [1]:
import numpy as np

In [2]:
# Load your data. Here we're creating random data.
coords = np.random.randn( 20000, 3 )
fields = np.random.random(size=coords[:,0].size)

## Format Data for Firefly

In [3]:
from firefly.data_reader import ArrayReader

Here are the [docs for ArrayReader](https://firefly.rcs.northwestern.edu/docs/reference/api/readers/firefly.data_reader.ArrayReader.html).

In [4]:
my_arrayReader = ArrayReader(
    coords,
    fields=fields,
    write_to_disk=False)

datadir is None, defaulting to /home/ageller/miniforge3/envs/firefly-wsl/lib/python3.11/site-packages/firefly/static/data/Data
Make sure each field_array (1) has a field_filter_flag (0), assuming True.
Make sure each field_array (1) has a field_colormap_flag (0), assuming True.
Make sure each field_array (1) has a field_radius_flag (0), assuming False.
PGroup_0 - 20000/20000 particles - 1 tracked fields


## Display Inline

In [5]:
from firefly.server import spawnFireflyServer,quitAllFireflyServers

In [6]:
# define the local port (typically anything in 5000 - 8000 range)
port = 5500

In [7]:
process = spawnFireflyServer(port, method = 'flask')

Waiting up to 10 seconds for background Firefly server to start.....Launching Firefly at: http://localhost:5500
from directory /home/ageller/miniforge3/envs/firefly-wsl/lib/python3.11/site-packages/firefly
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
done! Your server is available at - http://localhost:5500


In [8]:
from IPython.display import IFrame
url = f'http://localhost:{port:d}/combined'
IFrame(url, width=1000, height=500)

TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THI

In [9]:
# Send data to the server.
# Wait until it loads to run this command
my_arrayReader.sendDataViaFlask()

PGroup_0 - 20000/20000 particles - 1 tracked fields
Posting data on port 5500...======= receiving data from server ...
======= size of data 1062741
======= showing loader
======= sending data to viewer ...
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
/home/ageller/miniforge3/envs/firefly-wsl/lib/python3.11/site-packages/firefly/static/data/Data/DataPGroup_0000.json 1061162
/home/ageller/miniforge3/envs/firefly-wsl/lib/python3.11/site-packages/firefly/static/data/Data/DataSettings.json 746
/home/ageller/miniforge3/envs/firefly-wsl/lib/python3.11/site-packages/firefly/static/data/Data/filenames.json 93
/home/ageller/miniforge3/envs/firefly-wsl/lib/python3.11/site-packages/firefly/static/data/startup.json 45
======= done
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
data posted!


TESTING flask -- THIS NEEDS TO BE UPDATED ON pip
TESTING flask -- THIS NEEDS TO BE UPDATED ON pip


## Get the selected data in Python

(after using the selection tool)

In [ ]:
import requests

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# send a get request to receive the current settings from Firefly
# for larger amounts of data, you will need to increase the waitTime (in seconds) via params (see below; the default is 10s)
r = requests.get(url = f'http://localhost:{port:d}/get_selected_data', params = {'waitTime':60})
if r.status_code == 200:
    # success
    selection = r.json()
    # as a check (UPDATE THIS)
    print(selection['Gas']['Coordinates_flat'][:100]) 
else:
    print('Error: {}'.format(r.status_code), r.content)


In [ ]:
# plot x, y for the selected points
partsKeys = list(selection.keys())
part0 = selection[partsKeys[0]]

# UPDATE THIS
x = part0['Coordinates_flat'][0::3]
y = part0['Coordinates_flat'][1::3]

f, ax = plt.subplots()
ax.scatter(x[:1000],y[:1000])

### Quit the Firefly server

... this doesn't always work in a notebook ... you can also quit the server by resetting the kernel.

In [ ]:
return_code = quitAllFireflyServers()